In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [60]:
df = pd.read_csv("transactions.csv")
df

,Unnamed: 0,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,0,737265056,737265056,5000,5000.00,2016-08-13T14:27:32,98.55,Uber,US,US,...,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False
1,1,737265056,737265056,5000,5000.00,2016-10-11T05:05:54,74.51,AMC #191138,US,US,...,NaN,0.00,NaN,NaN,NaN,True,NaN,NaN,False,False
2,2,737265056,737265056,5000,5000.00,2016-11-08T09:18:39,7.47,Play Store,US,US,...,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False
3,3,737265056,737265056,5000,5000.00,2016-12-10T02:14:50,7.47,Play Store,US,US,...,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False
4,4,830329091,830329091,5000,5000.00,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,...,NaN,0.00,NaN,NaN,NaN,True,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,786358,732852505,732852505,50000,48904.96,2016-12-22T18:44:12,119.92,Lyft,US,US,...,NaN,1095.04,NaN,NaN,NaN,False,NaN,NaN,False,False
786359,786359,732852505,732852505,50000,48785.04,2016-12-25T16:20:34,18.89,hulu.com,US,US,...,NaN,1214.96,NaN,NaN,NaN,False,NaN,NaN,False,False
786360,786360,732852505,732852505,50000,48766.15,2016-12-27T15:46:24,49.43,Lyft,US,US,...,NaN,1233.85,NaN,NaN,NaN,False,NaN,NaN,False,False
786361,786361,732852505,732852505,50000,48716.72,2016-12-29T00:30:55,49.89,walmart.com,US,US,...,NaN,1283.28,NaN,NaN,NaN,False,NaN,NaN,False,False


In [62]:
row = df['isFraud']
df['label'] = 0
for i in range(len(row)):
    if row[i] == True:
        df['label'][i] = 1

C:\Users\shiva\AppData\Local\Temp\ipykernel_24964\669654030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = 1


In [63]:
set(df['transactionType'])

{'ADDRESS_VERIFICATION', 'PURCHASE', 'REVERSAL', nan}

In [70]:
numeric_columns = df.select_dtypes(include=[float, int]).columns
normalized_df = df.copy()
normalized_df[numeric_columns] = df[numeric_columns].apply(lambda x: x / x.max())
normalized_df

,Unnamed: 0,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,...,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud,label
0,0.000000,0.737715,0.737715,0.1,0.100000,2016-08-13T14:27:32,0.048992,Uber,US,US,...,0.000000,NaN,NaN,NaN,False,NaN,NaN,False,False,0.0
1,0.000001,0.737715,0.737715,0.1,0.100000,2016-10-11T05:05:54,0.037041,AMC #191138,US,US,...,0.000000,NaN,NaN,NaN,True,NaN,NaN,False,False,0.0
2,0.000003,0.737715,0.737715,0.1,0.100000,2016-11-08T09:18:39,0.003714,Play Store,US,US,...,0.000000,NaN,NaN,NaN,False,NaN,NaN,False,False,0.0
3,0.000004,0.737715,0.737715,0.1,0.100000,2016-12-10T02:14:50,0.003714,Play Store,US,US,...,0.000000,NaN,NaN,NaN,False,NaN,NaN,False,False,0.0
4,0.000005,0.830836,0.830836,0.1,0.100000,2016-03-24T21:04:46,0.035386,Tim Hortons #947751,US,US,...,0.000000,NaN,NaN,NaN,True,NaN,NaN,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,0.999995,0.733300,0.733300,1.0,0.978099,2016-12-22T18:44:12,0.059616,Lyft,US,US,...,0.023054,NaN,NaN,NaN,False,NaN,NaN,False,False,0.0
786359,0.999996,0.733300,0.733300,1.0,0.975701,2016-12-25T16:20:34,0.009391,hulu.com,US,US,...,0.025579,NaN,NaN,NaN,False,NaN,NaN,False,False,0.0
786360,0.999997,0.733300,0.733300,1.0,0.975323,2016-12-27T15:46:24,0.024573,Lyft,US,US,...,0.025976,NaN,NaN,NaN,False,NaN,NaN,False,False,0.0
786361,0.999999,0.733300,0.733300,1.0,0.974334,2016-12-29T00:30:55,0.024802,walmart.com,US,US,...,0.027017,NaN,NaN,NaN,False,NaN,NaN,False,False,0.0


In [71]:
dict1 = {'ADDRESS_VERIFICATION' : 0, 'PURCHASE' : 1, 'REVERSAL' : 0.5}
normalized_df['transactionTypeCat'] = 0

In [73]:
for i in range(len(normalized_df)):
    try:
        normalized_df['transactionTypeCat'][i] = dict1[normalized_df['transactionType'][i]]
    except:
        normalized_df['transactionTypeCat'][i] = normalized_df['transactionType'][i]

C:\Users\shiva\AppData\Local\Temp\ipykernel_24964\1342053841.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normalized_df['transactionTypeCat'][i] = dict1[normalized_df['transactionType'][i]]


In [79]:
used_df = normalized_df[['creditLimit','availableMoney', 'transactionAmount', 'currentBalance', 'transactionTypeCat', 'label']]
used_df = used_df.dropna()
x = used_df[['creditLimit','availableMoney', 'transactionAmount', 'currentBalance', 'transactionTypeCat']]
y = used_df[['label']]
print(set(x['transactionTypeCat']), len(x), len(y))

{0.0, 1.0, 0.5} 785665 785665


In [161]:
clf = RandomForestClassifier(max_depth=5, random_state=42, class_weight='balanced', n_estimators = 2500)

In [ ]:
clf.fit(x, y)

C:\Users\shiva\AppData\Local\Temp\ipykernel_24964\3201132111.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(x, y)


In [ ]:
X = clf.predict(x)
confusion_matrix(y, X)

In [ ]:
classifier = LogisticRegression(class_weight='balanced', random_state=42)

In [ ]:
classifier.fit(x, y)

In [ ]:
X = classifier.predict(x)
confusion_matrix(y, X)

In [ ]:
Prepared a model using machine learning to predict fraud transaction from a given set of credit card transactions